## Experiments on Instrument Detection Models

11/30/2020

Evaluation Metrics: F1 score, Expected Calibration Error (ECE)

In [1]:
# set up autoreload 
%load_ext autoreload
%autoreload 2

In [2]:
# define paths to datamodules 
dm_paths = {
    'audio': 
    '/home/hugo/CHONK/data/mdb-hop-0.25-chunk-1-AUGMENTED/splits-85-15', 
    
    'embeddings-512': 
    '/home/hugo/CHONK/data/mdb-hop-0.25-chunk-1-AUGMENTED-EMBEDDINGS-512/splits-85-15', 

    'embeddings-6144':
    '/home/hugo/CHONK/data/mdb-hop-0.25-chunk-1-AUGMENTED-EMBEDDINGS-6144/splits-85-15'}


In [3]:
import datetime
import os
now = datetime.datetime.now().strftime("%d.%m.%Y-%H.%M.%S")
log_dir = os.path.join('logs', f'experiment-{now}')
# define experiment
FIXED = dict(
    batch_size=128*3,
    num_workers=20,

    learning_rate=3e-4,
    weighted_cross_entropy=True,

    dropout=0.5,
    random_seed=20, 
    max_epochs=80, 
    log_dir=log_dir,
    
    version=0, 
    gpuid=-1
)

trials = [
    # EFFECTIVE RISK MINIMIZATION (NO MIXUP)
    dict(name='openl3-512-mlp-ERM',
        path_to_data=dm_paths['embeddings-512'],
        use_embeddings= True, 
        model_name='mlp-512',
        mixup=False,
        mixup_alpha=0), 

    dict(name='openl3-6144-mlp-ERM',
        path_to_data=dm_paths['embeddings-6144'],
        use_embeddings= True, 
        model_name='mlp-6144',
        mixup=False,
        mixup_alpha=0), 
    
#     dict(name='baseline-512-mlp-ERM', 
#          path_to_data=dm_paths['audio'],
#          batch_size=48*3,
#          use_embeddings=False, 
#          model_name='baseline-512', 
#          mixup=False, 
#          mixup_alpha=0),
    
    dict(name='baseline-6144-mlp-ERM', 
         path_to_data=dm_paths['audio'], 
         batch_size=48*3,
         use_embeddings=False, 
         model_name='baseline-6144', 
         mixup=False, 
         mixup_alpha=0),
    
    # MODELS WITH MIXUP (ALPHA = 0.2)
#     dict(name='openl3-512-mlp-MIXUP-alpha=0.2',
#         path_to_data=dm_paths['embeddings-512'],
#         use_embeddings= True, 
#         model_name='mlp-512',
#         mixup=True,
#         mixup_alpha=0.2),

    dict(name='openl3-6144-mlp-MIXUP-alpha=0.2',
        path_to_data=dm_paths['embeddings-6144'],
        use_embeddings= True, 
        model_name='mlp-6144',
        mixup=True,
        mixup_alpha=0.2),
    
    # MODELS WITH MIXUP (ALPHA = 0.4)
#     dict(name='openl3-512-mlp-MIXUP-alpha=0.4',
#         path_to_data=dm_paths['embeddings-512'],
#         use_embeddings= True, 
#         model_name='mlp-512',
#         mixup=True,
#         mixup_alpha=0.2),
    
    dict(name='openl3-6144-mlp-MIXUP-alpha=0.4',
        path_to_data=dm_paths['embeddings-6144'],
        use_embeddings= True, 
        model_name='mlp-6144',
        mixup=True,
        mixup_alpha=0.4)]

exp_dicts = [dict(FIXED) for t in trials]
for exp, t in zip(exp_dicts, trials):
    exp.update(t)



In [4]:
from instrument_recognition.task import load_datamodule, load_model, InstrumentDetectionTask, train_instrument_detection_model

def run_trial(exp_dict):

    # load the datamodule
    print(f'loading datamodule...')
    dm = load_datamodule(path_to_data=exp_dict['path_to_data'], 
                         batch_size=exp_dict['batch_size'], 
                         num_workers=exp_dict['num_workers'],
                         use_embeddings=exp_dict['use_embeddings'])
    
    # get classlist and number of classes
    classlist = dm.get_classes()
    num_output_units = len(classlist)
    print(f'classlist is: {classlist}')

    # load model
    print(f'loading model...')
    model = load_model(model_name=exp_dict['model_name'], 
                       output_units=num_output_units, 
                       dropout=exp_dict['dropout'])
    
    # build task
    print(f'building task...')
    task = InstrumentDetectionTask(model, dm, 
                            max_epochs=exp_dict['max_epochs'],
                            learning_rate=exp_dict['learning_rate'], 
                            weighted_cross_entropy=exp_dict['weighted_cross_entropy'], 
                            mixup=exp_dict['mixup'],
                            mixup_alpha=exp_dict['mixup_alpha'], 
                            log_epoch_metrics=True)
    
    # run train fn and get back test results
    print(f'running task')
    result = train_instrument_detection_model(task, 
                                    name=exp_dict['name'], 
                                    version=exp_dict['version'], 
                                    gpuid=exp_dict['gpuid'], 
                                    max_epochs=exp_dict['max_epochs'],
                                    random_seed=exp_dict['random_seed'], 
                                    log_dir=exp_dict['log_dir'],)
    
    result = result[0]

    result['done'] = True

    return result

/home/hugo/lab/venv/lib/python3.6/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '
/home/hugo/lab/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hugo/lab/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/

In [5]:
%%capture cap
import traceback 
import pandas as pd

scoreboard = []
for exp in exp_dicts:
    print(100*'-', '\n', 100*'-')
    try:
        results = run_trial(exp)
    except:
        print(f'exp: {exp["name"]} failed.')
        traceback.print_exc(limit=None, file=None, chain=True)
        results = {'error': True, 'done': False}
        
    score = dict(exp)
    score.update(results)
    scoreboard.append(score)
    
scoreboard_df = pd.DataFrame(scoreboard)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name       | Type        | Params
-------------------------------------------
0 | model      | MLP512      | 69 K  
1 | model.fc   | Sequential  | 69 K  
2 | model.fc.0 | BatchNorm1d | 1 K   
3 | model.fc.1 | Linear      | 65 K  
4 | model.fc.2 | ReLU        | 0     
5 | model.fc.3 | Dropout     | 0     
6 | model.fc.4 | BatchNorm1d | 256   
7 | model.fc.5 | Linear      | 2 K   
Epoch 0: loss/val reached 1.29530 (best 1.29530), saving model to /home/hugo/lab/mono_music_sed/instrument_recognition/notebooks/logs/experiment-01.12.2020-01.54.52/openl3-512-mlp-ERM/version_0/checkpoints/epoch=00-loss_val=1.29.ckpt as top 3
Epoch 1: loss/val reached 1.33528 (best 1.29530), saving model to /home/hugo/lab/mono_music_sed/instrument_recognition/notebooks/logs/experiment-01.12.2020-01.54.52/openl3-512-mlp-ERM/version_0/checkpoints/epoch=01-loss_val=1.34.ckpt as top 3
Epoch 2:

Epoch 8: loss/val was not in top 3
Epoch 9: loss/val was not in top 3
Epoch 10: loss/val was not in top 3
Epoch 11: loss/val was not in top 3
Epoch 12: loss/val was not in top 3
Epoch 13: loss/val was not in top 3
Epoch 14: loss/val was not in top 3
Epoch 15: loss/val was not in top 3
Epoch 16: loss/val was not in top 3
Epoch 17: loss/val was not in top 3
Epoch 18: loss/val was not in top 3
Epoch 19: loss/val was not in top 3
Epoch 20: loss/val was not in top 3
Epoch 21: loss/val was not in top 3
Epoch 22: loss/val was not in top 3
Epoch 23: loss/val was not in top 3
Epoch 24: loss/val was not in top 3
Epoch 25: loss/val was not in top 3
Epoch 26: loss/val was not in top 3
Epoch 27: loss/val was not in top 3
Epoch 28: loss/val was not in top 3
Epoch 29: loss/val was not in top 3
Epoch 30: loss/val was not in top 3
Epoch 31: loss/val was not in top 3
Epoch 32: loss/val was not in top 3
Epoch 33: loss/val was not in top 3
Epoch 34: loss/val was not in top 3
Epoch 35: loss/val was not in 

Epoch 2: loss/val reached 1.30627 (best 1.29144), saving model to /home/hugo/lab/mono_music_sed/instrument_recognition/notebooks/logs/experiment-01.12.2020-01.54.52/openl3-6144-mlp-MIXUP-alpha=0.2/version_0/checkpoints/epoch=02-loss_val=1.31.ckpt as top 3
Epoch 3: loss/val reached 1.29252 (best 1.29144), saving model to /home/hugo/lab/mono_music_sed/instrument_recognition/notebooks/logs/experiment-01.12.2020-01.54.52/openl3-6144-mlp-MIXUP-alpha=0.2/version_0/checkpoints/epoch=03-loss_val=1.29.ckpt as top 3
Epoch 4: loss/val reached 1.24562 (best 1.24562), saving model to /home/hugo/lab/mono_music_sed/instrument_recognition/notebooks/logs/experiment-01.12.2020-01.54.52/openl3-6144-mlp-MIXUP-alpha=0.2/version_0/checkpoints/epoch=04-loss_val=1.25.ckpt as top 3
Epoch 5: loss/val reached 1.29132 (best 1.24562), saving model to /home/hugo/lab/mono_music_sed/instrument_recognition/notebooks/logs/experiment-01.12.2020-01.54.52/openl3-6144-mlp-MIXUP-alpha=0.2/version_0/checkpoints/epoch=05-loss

Epoch 2: loss/val reached 1.34252 (best 1.32370), saving model to /home/hugo/lab/mono_music_sed/instrument_recognition/notebooks/logs/experiment-01.12.2020-01.54.52/openl3-6144-mlp-MIXUP-alpha=0.4/version_0/checkpoints/epoch=02-loss_val=1.35.ckpt as top 3
Epoch 3: loss/val reached 1.34033 (best 1.32370), saving model to /home/hugo/lab/mono_music_sed/instrument_recognition/notebooks/logs/experiment-01.12.2020-01.54.52/openl3-6144-mlp-MIXUP-alpha=0.4/version_0/checkpoints/epoch=03-loss_val=1.34.ckpt as top 3
Epoch 4: loss/val was not in top 3
Epoch 5: loss/val reached 1.32594 (best 1.32370), saving model to /home/hugo/lab/mono_music_sed/instrument_recognition/notebooks/logs/experiment-01.12.2020-01.54.52/openl3-6144-mlp-MIXUP-alpha=0.4/version_0/checkpoints/epoch=05-loss_val=1.33.ckpt as top 3
Epoch 6: loss/val reached 1.31711 (best 1.31711), saving model to /home/hugo/lab/mono_music_sed/instrument_recognition/notebooks/logs/experiment-01.12.2020-01.54.52/openl3-6144-mlp-MIXUP-alpha=0.4/

In [6]:
%store 

Stored variables and their in-db values:


In [7]:
scoreboard_df.to_csv(os.path.join(log_dir, 'scoreboard.csv'))
with open(os.path.join(log_dir, 'output.txt'),  'w') as f:
    f.write(cap.stdout)